# 📊 Dataset difficulty - Jupyter notebook

## Augmentation

standard transformation only creates new columens *sentence1* and *sentence 2* for standardization, null transformation pass empty strings to these variables

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dataset_difficulty.augment import FEVERStandardTransformation, FEVERNullTransformation

FEVERStandardTransformation("ctu-aic/csfever_v2","/home/mlynatom/data/dataset_difficulty/augmentation/csfever_v2/", dataset_subset="original_nli").transform()
FEVERNullTransformation("ctu-aic/csfever_v2","/home/mlynatom/data/dataset_difficulty/augmentation/csfever_v2/", dataset_subset="original_nli").transform()

## Train Models

Do using other finetuning scripts

## Estimate V-usable Info

In [2]:
from dataset_difficulty.v_info import v_info

DATASET_PATH = "/home/mlynatom/data/dataset_difficulty/augmentation/csfever_v2/"
SPLITS = ["train", 
          "dev", 
          "test"
          ]

for split in SPLITS:
       v_info(data_fn=f"{DATASET_PATH}fever_{split}_std.csv", 
              model="/home/mlynatom/models/peft-lora-xlm-roberta-large-squad2-csfever_v2-original_nli-r8-alpha16_bias-none", 
              null_data_fn=f"{DATASET_PATH}fever_{split}_null.csv",
              null_model="/home/mlynatom/models/peft-lora-xlm-roberta-large-squad2-csfever_v2_null-original_nli-r8-alpha16_bias-none",
              tokenizer="ctu-aic/xlm-roberta-large-squad2-csfever_v2-f1",
              out_fn=f"/home/mlynatom/data/dataset_difficulty/PVI/csfever_v2_{split}.csv",
              input_key="sentence1",
              input_key2="sentence2",
              use_lora=True)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at deepset/xlm-roberta-large-squad2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 10/1190 [00:16<28:52,  1.47s/it] /home/mlynatom/venvs/py3.9.6/lib/python3.9/site-packages/transformers/pipelines/base.py:1083: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 1190/1190 [28:08<00:00,  1.42s/it]
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at deepset/xlm-roberta-large-squad2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You shou

## PVI Analysis

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("/home/mlynatom/data/dataset_difficulty/PVI/csfever_v2_test.csv")

In [6]:
sorted_data = data.sort_values(by="PVI", ascending=True)

In [8]:
sorted_data.head(n=20)

,Unnamed: 0,id,label,claim,evidence,sentence1,sentence2,H_yb,H_yx,correct_yx,predicted_label,PVI
974,974,64142,0,"K dubnu 2016 se prodalo 4,7 milionů digitálníc...",The Fame. The Fame je první studiové album ame...,"K dubnu 2016 se prodalo 4,7 milionů digitálníc...",The Fame. The Fame je první studiové album ame...,0.975885,11.271565,False,2,-10.295680
4368,4368,188568,1,Pitch Perfect 2 se točí kolem skutečné univerz...,Ladíme 2. Ladíme 2 je americká hudební komedie...,Pitch Perfect 2 se točí kolem skutečné univerz...,Ladíme 2. Ladíme 2 je americká hudební komedie...,2.620849,12.907406,False,2,-10.286556
3704,3704,188576,0,Pitch Perfect 2 se točí kolem fiktivní univerz...,Ladíme 2. Ladíme 2 je americká hudební komedie...,Pitch Perfect 2 se točí kolem fiktivní univerz...,Ladíme 2. Ladíme 2 je americká hudební komedie...,0.975885,10.558119,False,2,-9.582234
2124,2124,108486,1,Sky UK obsluhuje Irsko.,Sky TV. Sky TV (hojně známá jen jako Sky) je n...,Sky UK obsluhuje Irsko.,Sky TV. Sky TV (hojně známá jen jako Sky) je n...,2.620849,12.025472,False,2,-9.404623
7095,7095,188582,1,Pitch Perfect 2 se soustředí na hornické město.,Ladíme 2. Ladíme 2 je americká hudební komedie...,Pitch Perfect 2 se soustředí na hornické město.,Ladíme 2. Ladíme 2 je americká hudební komedie...,2.620849,11.522157,False,2,-8.901308
5759,5759,188571,0,Pitch Perfect 2 se zaměřuje na fiktivní vzdělá...,Ladíme 2. Ladíme 2 je americká hudební komedie...,Pitch Perfect 2 se zaměřuje na fiktivní vzdělá...,Ladíme 2. Ladíme 2 je americká hudební komedie...,0.975885,9.777429,False,2,-8.801544
7459,7459,188569,0,Pitch Perfect 2 se soustředí na a cappella pěv...,Ladíme 2. Ladíme 2 je americká hudební komedie...,Pitch Perfect 2 se soustředí na a cappella pěv...,Ladíme 2. Ladíme 2 je americká hudební komedie...,0.975885,9.658117,False,2,-8.682232
3836,3836,188570,0,Pitch Perfect 2 se soustředí na pěveckou skupinu.,Ladíme 2. Ladíme 2 je americká hudební komedie...,Pitch Perfect 2 se soustředí na pěveckou skupinu.,Ladíme 2. Ladíme 2 je americká hudební komedie...,0.975885,9.391499,False,2,-8.415614
5931,5931,120290,0,TV Land má televizní seriál na TV Land.,"Znovu 20. Znovu 20 (čti ""Znovu dvacítkou"", v a...",TV Land má televizní seriál na TV Land.,"Znovu 20. Znovu 20 (čti ""Znovu dvacítkou"", v a...",0.975885,9.223727,False,2,-8.247841
1000,1000,51683,1,Toy Story 2 bylo vydáno v Severní Koreji.,Walt Disney Pictures. Walt Disney Pictures je ...,Toy Story 2 bylo vydáno v Severní Koreji.,Walt Disney Pictures. Walt Disney Pictures je ...,2.620849,10.521515,False,2,-7.900665


In [12]:
filtered_data=sorted_data[sorted_data["correct_yx"]][sorted_data["PVI"] < 0]

/tmp/ipykernel_3751470/4191321605.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data=sorted_data[sorted_data["correct_yx"]][sorted_data["PVI"] < 0]


In [13]:
filtered_data

,Unnamed: 0,id,label,claim,evidence,sentence1,sentence2,H_yb,H_yx,correct_yx,predicted_label,PVI
3192,3192,22609,0,Therasia je založena na sopečné skupině ostrov...,Thirasia. Thirasia () je ostrov v Egejském moř...,Therasia je založena na sopečné skupině ostrov...,Thirasia. Thirasia () je ostrov v Egejském moř...,0.975885,1.527626,True,0,-0.551741
4351,4351,83509,0,Celoživotní přítelkyní Trumana Capoteho byla H...,Truman Capote. Truman Capote vlastním jménem T...,Celoživotní přítelkyní Trumana Capoteho byla H...,Truman Capote. Truman Capote vlastním jménem T...,0.975885,1.212529,True,0,-0.236644
1908,1908,98606,0,Bonobové byli odděleni od obydlí běžných šimpa...,"Šimpanz bonobo. Šimpanz bonobo (""Pan paniscus""...",Bonobové byli odděleni od obydlí běžných šimpa...,"Šimpanz bonobo. Šimpanz bonobo (""Pan paniscus""...",0.975885,1.204089,True,0,-0.228203
2347,2347,1917,0,Současný vévoda z Yorku je od svého rozvodu v ...,Vévoda z Yorku. Vévoda z Yorku (anglicky Duke ...,Současný vévoda z Yorku je od svého rozvodu v ...,Vévoda z Yorku. Vévoda z Yorku (anglicky Duke ...,0.975885,1.199975,True,0,-0.224090
5063,5063,174510,0,Red Bull Racing sídlí ve Spojeném království.,Red Bull Racing. Red Bull Racing (plným názvem...,Red Bull Racing sídlí ve Spojeném království.,Red Bull Racing. Red Bull Racing (plným názvem...,0.975885,1.155574,True,0,-0.179689
7370,7370,116423,0,Vandalové založili království ve Španělsku a s...,Vandalové. Vandalové byli velký východogermáns...,Vandalové založili království ve Španělsku a s...,Vandalové. Vandalové byli velký východogermáns...,0.975885,1.143860,True,0,-0.167974
4915,4915,157878,0,Bonobové byli odděleni od předků běžného šimpa...,"Šimpanz bonobo. Šimpanz bonobo (""Pan paniscus""...",Bonobové byli odděleni od předků běžného šimpa...,"Šimpanz bonobo. Šimpanz bonobo (""Pan paniscus""...",0.975885,1.124664,True,0,-0.148779
6007,6007,26381,0,Bonobové žijí jižně od řeky.,"Šimpanz bonobo. Šimpanz bonobo (""Pan paniscus""...",Bonobové žijí jižně od řeky.,"Šimpanz bonobo. Šimpanz bonobo (""Pan paniscus""...",0.975885,1.123129,True,0,-0.147244
7110,7110,162837,0,Islámábád má patnáct univerzit.,Islámábád. Islámábád () je hlavní město Pákist...,Islámábád má patnáct univerzit.,Islámábád. Islámábád () je hlavní město Pákist...,0.975885,1.108659,True,0,-0.132774
1728,1728,57008,0,Nymfomanka (film) je spolu s Antikristem součá...,Nymfomanka (film). Nymfomanka (původním názvem...,Nymfomanka (film) je spolu s Antikristem součá...,Nymfomanka (film). Nymfomanka (původním názvem...,0.975885,1.077342,True,0,-0.101457


In [14]:
filtered_data2=sorted_data[sorted_data["correct_yx"] != True][sorted_data["PVI"] > 0]

/tmp/ipykernel_3751470/492028832.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data2=sorted_data[sorted_data["correct_yx"] != True][sorted_data["PVI"] > 0]


In [15]:
filtered_data2

,Unnamed: 0,id,label,claim,evidence,sentence1,sentence2,H_yb,H_yx,correct_yx,predicted_label,PVI
1615,1615,46833,2,Yvonne Strahovski pracovala na dobovém dramatu...,Yvonne Strahovski. Yvonne Jaqueline Strahovski...,Yvonne Strahovski pracovala na dobovém dramatu...,Yvonne Strahovski. Yvonne Jaqueline Strahovski...,1.603840,1.600551,False,0,0.003288
5057,5057,160481,1,"Jednou ze tří výkonných producentek Buffy, pře...","Buffy, přemožitelka upírů. Buffy, přemožitelka...","Jednou ze tří výkonných producentek Buffy, pře...","Buffy, přemožitelka upírů. Buffy, přemožitelka...",2.620849,2.614328,False,0,0.006522
3005,3005,70393,1,Linkin Park měli pauzu od roku 2000 do roku 2005.,Linkin Park. Linkin Park je americká rocková k...,Linkin Park měli pauzu od roku 2000 do roku 2005.,Linkin Park. Linkin Park je americká rocková k...,2.620849,2.613287,False,0,0.007563
3120,3120,115898,1,Linkin Park se poprvé dostali do žebříčku Bill...,Linkin Park. Linkin Park je americká rocková k...,Linkin Park se poprvé dostali do žebříčku Bill...,Linkin Park. Linkin Park je americká rocková k...,2.620849,2.613248,False,0,0.007602
3176,3176,146359,1,Pochody v Selmě a Montgomery byly součástí šir...,Pochody ze Selmy do Montgomery. Pochody ze Sel...,Pochody v Selmě a Montgomery byly součástí šir...,Pochody ze Selmy do Montgomery. Pochody ze Sel...,2.620849,2.610815,False,2,0.010035
...,...,...,...,...,...,...,...,...,...,...,...,...
2541,2541,45604,1,Argentina je rozdělena na 48 měst.,"Argentina. Argentina, plným názvem Argentinská...",Argentina je rozdělena na 48 měst.,"Argentina. Argentina, plným názvem Argentinská...",2.620849,1.095002,False,0,1.525847
72,72,113688,1,Romové vymřeli zhruba před 1000 lety.,"Romové. Romové jsou etnikum, jehož kořeny saha...",Romové vymřeli zhruba před 1000 lety.,"Romové. Romové jsou etnikum, jehož kořeny saha...",2.620849,1.080010,False,0,1.540839
6583,6583,102636,1,Poslední vichřice ve Stanley Parku byla v roce...,"Stanley Park. Stanley Park (v překladu ""Stanle...",Poslední vichřice ve Stanley Parku byla v roce...,"Stanley Park. Stanley Park (v překladu ""Stanle...",2.620849,1.045267,False,0,1.575582
1541,1541,72182,1,Eurotas (řeka) je v Itálii.,Evrotas. Evrotas je řeka na řeckém poloostrově...,Eurotas (řeka) je v Itálii.,Evrotas. Evrotas je řeka na řeckém poloostrově...,2.620849,1.033816,False,0,1.587033


### Remap values

In [ ]:
label_dict = {0: "SUPPORTS",
              1: "REFUTES",
              2: "NOT ENOUGH INFO"}

In [ ]:
replaced_data = data.replace({"predicted_label": label_dict})

### Load annotated data

In [ ]:
annotated_data = pd.read_csv("data/annotated_fcheck/fever_pair_1%_annotated.csv")

In [ ]:
annotated_data.head()

### Merge human_labels

In [ ]:
pd.merge(data, annotated_data[["id", "human_label"]], on="id", how="inner")

### Compute and plot correlation

correlation between PVI and score

correlation between mislabeled (from human_labels) and PVI

## Deleted data analysis using PVI

### mark which were deleted

### investigate their PVI's (possibly plot)